In [1]:
import pandas as pd
import socket
import requests 
import re
import builtwith
from bs4 import BeautifulSoup
from difflib import SequenceMatcher

In [2]:
ecommerce_services_list = "WooCommerce Shopify Magento PrestaShop OpenCart Wix Bigcommerce Shopware osCommerce Amazon"

In [3]:
social_media_keywords = ["instagram", "facebook","twitter"]
black_list_keywords = ["nightclub","ecommerce","e-commerce"]
positive_keywords = ["organic","recycle","recycling","health","artisan", "curate", "sustainable", "Michelin", "awards","local"]

In [4]:
list_stores = ['store',
 'shop',
 'outlet',
 'mart',
 'boutique',
 'emporium',
 'bazaar',
 'market',
 'showroom',
 'co-op',
 'mall',
 'galleria',
 'exchange',
 'bargains',
 'botique']

In [5]:
agent = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'}
 

In [6]:
def generateCustomerListDataFrame(add_samples=False):
    raw_company_data = {'Company Name':[],'Company URL':[],'Has Ecommerce':False}
    df = pd.DataFrame(raw_company_data)
    

    if (add_samples==True):
        one = ("Joe's Cafe","http://www.joescafebrighton.com/", "not")
        two = ("Moksha Cafe", "https://mokshacaffe.co.uk/", "not")
        three = ("Cafe Coho" ,"https://www.cafecoho.co.uk/", "is")
        four = ("Lovefit","https://www.lovefit.co.uk/","not")
        five = ("Porter and York","https://porterandyork.com/","is")
        six = ("Dineamic", "https://www.dineamic.com.au/","is")
        seven = ("Haus London","https://hauslondon.com/","is")
        eight = ("The Critical Slide Society", "https://www.thecriticalslidesociety.eu/","is")
        nine = ("Studio Proper","https://www.studioproper.com/","is")
        ten = ("Tluxe","https://www.tluxe.com/", "is")
        total = [one,two,three,four,five,six,seven,eight,nine,ten]
        
        for number in total:
            has_e = False
            if number[2] == "is":
                has_e = True
            df = df.append({'Company Name':number[0],'Company URL': number[1],'Has Ecommerce':has_e},ignore_index=True)
    

    
    return df

In [7]:
# def addSamplesDf(df,our_company_name,our_company_url, has_ecommerce):
    
#     return df.append({'Company Name':our_company_name,'Company URL': our_company_url,'Has Ecommerce':has_ecommerce},ignore_index=True)
    

In [8]:
def analyseCustomerListDataFrame(df,add_colours=False):
    
    df['Text from Website'] = df['Company URL'].apply(getSoupfromURL)
    ### Here is the Soup
    
    

    df['Ecommerce Tags Found Score'] = df['Text from Website'].apply(getEcommerceTagScore)
    #
    df['Ecommerce IPs Found Score'] = df['Company URL'].apply(getEcommerceIPsScore)
    df["Builtwith's Ecommerce-Detection Score"] = df['Company URL'].apply(getBuiltWithEcommerceScore)
    df['Ecommerce Domain URL Component Score'] = df['Company URL'].apply(checkURLComponents)
    df['Ecommerce Detection Total Score'] = df['Ecommerce Tags Found Score'] +  df['Ecommerce IPs Found Score'] + df["Builtwith's Ecommerce-Detection Score"]
    
    df= getKeyWordsScore(df)
    
    
    #df['Social Media Keywords'] = 
    #Positive Key Words 
    #df['Black List Keywords']
#     if (add_colours == True):
#         return applyColouring()
        
    return df.drop('Text from Website',axis=1)

In [9]:
def getSoupfromURL(url):
    return BeautifulSoup(requests.get(url,headers=agent).content).text

In [10]:
def generateEcommerceListDataFrame():
    raw_ecommerce_company_data = {'Ecommerce Service Name':[],'Ecommerce Service URLs':[]}
    ecommerce_df = pd.DataFrame(raw_ecommerce_company_data)

    for service in ecommerce_services_list.split():
        ecommerce_df = ecommerce_df.append({'Ecommerce Service Name':service,'Ecommerce Service URLs':getURL(service)},ignore_index=True)
    return ecommerce_df

In [11]:
def similar(a, b):
    # this gets the similarity between two key words
    return SequenceMatcher(None, a, b).ratio()

In [12]:
def getURL(search_query,link_limit = 100,correlation_tolerance=0.6):
    
    #This gets the first set of URLs that correspond to a service.
    # The use is that we can iteratively go through a list of Ecommerce Services, and quickly find websites that relate to them
    # the correlation tolerance needs to be quite specific because the search results may not always be super relevant
  
    domain_list = ""
    ecommerce_likely_domains = ""
    
    page = requests.get(f"https://www.google.co.uk/search?q={search_query}")
    soup = BeautifulSoup(page.content)
    links = soup.findAll(search_query)
 
        

    for link in soup.find_all("a",href=re.compile("(?<=/url\?q=)(htt.*://.*)")):
        link = re.split(":(?=http)",link["href"].replace("/url?q=",""))
        domain_list += link[0].split("//")[-1].split("/")[0] + ", "
    
    domain_list = domain_list.split(",")
    
    for x in range(0,len(domain_list)):
        compared = similar(search_query,domain_list[x])
        if (compared> correlation_tolerance) or (search_query.lower() in domain_list[x].lower()):
             ecommerce_likely_domains += domain_list[x] + ", "
    
            
    
    return ' '.join(set(ecommerce_likely_domains.split()))
    

In [13]:
def addClientToDataFrame(our_company_name,our_company_url, has_ecommerce):
    return df.append({'Our Company Name':our_company_name,'Our Company URL': our_company_url,'Has Ecommerce':has_ecommerce},ignore_index=True)
    

In [14]:
def checkURLComponents(url):
    just_domain_body=False
    
    domain_body_score = 0
    domain_suffix_score = 0
    domain_tail_score = 0
    
#     domain_ip = "null"
    
    

    for word in list_stores:
        if word.lower() in url.lower():
                domain_body_score +=1
                break

    
#     if (just_domain_body == True):
#         return domain_body_score
#     else:
#         url = url.replace("https://","").replace("http://","").replace("/","").replace("www.","")
#         print(url)
        
      


#         for word in list_stores:
#             print("working on:")
#             print("www." + url.split(".")[0] + "." + word)
#             print("www." + url + "/" + word)
   

#             try:
              
#                 domain_ip = socket.gethostbyname("www." + url + "/" + word)
#                 print(domain_ip + socket.gethostbyname(url + "/" + word))
#                 if (socket.gethostbyname(url + "/" + word)):
#                     print("yes")
#                 if (domain_ip != "null"):
#                     print(url + word)
#                     domain_suffix_score +=1
#                     break
#             except:
#                 pass
                
#             try:
               
#                 domain_ip = "null"
#                 domain_ip = socket.gethostbyname("www." + url.split(".")[0] + "." + word)
              
                
        
#                 if (domain_ip != "null"):
#                     domain_tail_score += 1 
#                     print(url + "found a match with " + " " +word)
#                     break


#             except:
#                 pass
                
#         print("db " + str(domain_body_score) + "dss " + str(domain_suffix_score) + " dts " + str(domain_tail_score))
    return (domain_body_score + domain_suffix_score + domain_tail_score)


In [15]:
def getEcommerceTagScore(soup):
    
    score = 0
    
    for row in ecommerce_df[['Ecommerce Service Name','Ecommerce Service URLs']].iterrows():
        for item in row[1]:
            if item.lower() in soup.lower():
             
                score += 1
                break


        # this means that the appearance of each company, whether a keyword or url, will only increment score by one even if they're both added
     
    return score
        
                
                


In [16]:
def getBuiltWithEcommerceScore(url):
    score = 0
    builtwith_analysis = builtwith.builtwith(url)
    
    builtwith_as_text = ''.join('{}{}'.format(key, val) for key, val in builtwith_analysis.items())
    
    ## possibly include more general keywords like ecommerce
  
    
    for row in ecommerce_df[['Ecommerce Service Name','Ecommerce Service URLs']].iterrows():
        for item in row[1]:
            if item.lower() in builtwith_as_text.lower():
                score += 1
       
              
                break
    return score
    

In [17]:
def getEcommerceIPsScore(desired_domain):
    
    score = 0
    desired_domain = desired_domain.replace("https://","").replace("http://","").replace("/","")
 
    
    try:
        domain_ip = socket.gethostbyname(desired_domain)
     
        domain_lookup = "https://db-ip.com/"
    
        agent = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'}
        #The agent parameter is needed to create a "user" and to avoid any bot banning
    
    
        page = requests.get(domain_lookup+domain_ip,headers=agent)
        soup = BeautifulSoup(page.content)

    
        for row in ecommerce_df[['Ecommerce Service Name','Ecommerce Service URLs']].iterrows():
            for item in row[1]:
                if item.lower() in soup.text.lower():

                    score += 1
                    break


        # this means that the appearance of each company, whether a keyword or url, will only increment score by one even if they're both added
    except:
        score = -1
    return score
        

In [18]:
def getKeyWordsScore(df):
 
     
    
    
    def getSocialMediaScore(social_media_content):
        social_media_score = 0
        for word in social_media_keywords:
            if word.lower() in cleanText(social_media_content):
                social_media_score+=2
            if (word == "twitter"):
                social_media_score+=4 
        return social_media_score
    
    
    def getPositiveWordScore(positive_content):
        positive_word_score = 0
        for word in positive_keywords:
            if word.lower() in cleanText(positive_content):
                positive_word_score +=3
            if positive_word_score >= 10:
                positive_word_score = 10
        return positive_word_score 
    
    def getNegativeWordScore(negative_content):
        negative_word_score = 0
        for word in black_list_keywords:
            if word.lower() in cleanText(negative_content):
                negative_word_score+=1
        return negative_word_score
    
    
    
    df['Social_Media_Score'] = df['Text from Website'].apply(getSocialMediaScore)
    df['Positive_Keyword_Score'] = df['Text from Website'].apply(getPositiveWordScore)
    df['Blacklist_Keyword_Score'] = df['Text from Website'].apply(getNegativeWordScore)
    
    return df

In [19]:
def cleanText(text):
    
    #used for natural language processing such as Key Word detection
    
    from string import punctuation
    text = text.lower().replace(".", ". ")
    text = ''.join(char for char in text if char not in punctuation)
    ##update to include the punctuation particular to online websites e.g TM

    
    return text

In [20]:
ecommerce_df = generateEcommerceListDataFrame()
df = generateCustomerListDataFrame(add_samples=True)
df = analyseCustomerListDataFrame(df,add_colours=True)

In [21]:
df.head(100)

,Company Name,Company URL,Has Ecommerce,Ecommerce Tags Found Score,Ecommerce IPs Found Score,Builtwith's Ecommerce-Detection Score,Ecommerce Domain URL Component Score,Ecommerce Detection Total Score,Social_Media_Score,Positive_Keyword_Score,Blacklist_Keyword_Score
0,Joe's Cafe,http://www.joescafebrighton.com/,False,0,0,0,0,0,4,6,0
1,Moksha Cafe,https://mokshacaffe.co.uk/,False,1,0,0,0,1,8,6,0
2,Cafe Coho,https://www.cafecoho.co.uk/,True,1,0,1,0,2,8,3,0
3,Lovefit,https://www.lovefit.co.uk/,False,0,0,0,0,0,4,3,0
4,Porter and York,https://porterandyork.com/,True,1,0,1,0,2,8,0,0
5,Dineamic,https://www.dineamic.com.au/,True,2,1,1,0,4,10,6,1
6,Haus London,https://hauslondon.com/,True,2,1,1,0,4,8,0,1
7,The Critical Slide Society,https://www.thecriticalslidesociety.eu/,True,2,1,1,0,4,10,6,0
8,Studio Proper,https://www.studioproper.com/,True,2,1,1,0,4,8,3,1
9,Tluxe,https://www.tluxe.com/,True,2,1,1,0,4,8,0,1
